In [7]:
# ===== LiCoO2 CASCI (active-space FCI) with auto-shrink =====
# - Basis: STO-3G
# - Geometry from (C, L, theta) params
# - Active space: start from HOMO±K, score by Co-3d / O-2p character, cap by max_norb
# - If FCI dimension estimate too large, auto-shrink further
# - Robust to PySCF version: don't unpack kernel() return; read from attributes

import numpy as np
from math import comb
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / Basis / Spin ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"   # requested
mol.charge = 0         # adjust if needed
mol.spin   = 0         # 2S; singlet assumed. For open-shell, set properly and use UCASCI below
mol.max_memory = 20000
mol.build()

# ---------- 2) SCF (tight) ----------
mf = scf.RHF(mol) if mol.spin == 0 else scf.UHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) AO→MO projected character: Co-3d / O-2p ----------
def ao_ids_by_atom_L(mol, atom_sym: str, L_letter: str):
    ids = []
    for i, lab in enumerate(mol.ao_labels()):  # e.g., "Co 1 3dxy", "O 2 2px"
        parts = lab.split()
        atom_ok = parts[0].lower().startswith(atom_sym.lower())
        L_ok = (' '+L_letter) in (' '+lab) or parts[-1].lower().startswith(L_letter)
        if atom_ok and L_ok:
            ids.append(i)
    return ids

co_d_ao = ao_ids_by_atom_L(mol, "Co", "d")
o_p_ao  = ao_ids_by_atom_L(mol, "O",  "p")

# Use alpha mo_coeff if UHF; RHF has single set
mo = mf.mo_coeff if mol.spin == 0 else mf.mo_coeff[0]
nmo = mo.shape[1]

def proj_weight(ao_list):
    if not ao_list:
        return np.zeros(nmo)
    sub = mo[ao_list, :]             # (n_sel_AO, nmo)
    return np.sum(sub**2, axis=0)    # per-MO sum of squared AO contributions

w_co3d = proj_weight(co_d_ao)
w_o2p  = proj_weight(o_p_ao)

# ---------- 4) Active-space selection with auto shrink ----------
# Config knobs (tune by RAM/time)
K_window   = 12         # start from HOMO±K
max_norb   = 16        # cap active orbitals (e.g., 10~16)
max_detdim = 5_000_000  # cap on estimated FCI determinant space size

# Occupations (RHF or average if UHF)
occ = mf.mo_occ if np.ndim(mf.mo_occ) == 1 else 0.5*(mf.mo_occ[0] + mf.mo_occ[1])
occ = np.asarray(occ)
nmo = occ.size

# HOMO index (safe fallback if no occ>1.0)
homo = np.where(occ > 1.0)[0].max() if np.any(occ > 1.0) else nmo // 2

# Start from HOMO±K
cand = list(range(max(0, homo-K_window), min(nmo, homo+K_window+1)))

# Score = Co-3d weight + 0.5*O-2p weight + small bonus for proximity to HOMO
proximity = np.array([-abs(i - homo) for i in range(nmo)], dtype=float)
score = w_co3d + 0.5*w_o2p + 0.01*proximity

# Pick top by score within candidate window
cand_sorted = sorted(cand, key=lambda i: score[i], reverse=True)
active_idx  = sorted(cand_sorted[:max_norb])

# Light occupancy filter to avoid fully-filled/empty extremes (optional)
low, high = 0.05, 1.95
filtered = [i for i in active_idx if (occ[i] > low and occ[i] < high)]
if filtered:
    active_idx = sorted(filtered)

# Electron count in active space (round HF occupations)
nelecas = int(round(float(np.sum(occ[active_idx]))))
ncas    = len(active_idx)

# Estimate FCI dimension; if too large, drop lowest-score MOs until under limit
# singlet assumption for estimate (adjust if open-shell)
nalpha = nelecas // 2
nbeta  = nelecas - nalpha

def est_dim(norb, na, nb):
    try:
        return comb(norb, na) * comb(norb, nb)
    except ValueError:
        return float('inf')

while est_dim(ncas, nalpha, nbeta) > max_detdim and ncas > 2:
    # drop the currently selected MO with the smallest score
    worst_local = min(active_idx, key=lambda i: score[i])
    active_idx.remove(worst_local)
    ncas = len(active_idx)

dim_est = est_dim(ncas, nalpha, nbeta)

print(f"[CAS pick] ncas={ncas}, nelecas={nelecas}, dim≈{dim_est:,}")
print(f"[CAS idx] {active_idx}")
print("[Diag] occ(active):", [float(occ[i]) for i in active_idx])
print("[Diag] Co-3d weight(active):", [float(w_co3d[i]) for i in active_idx])
print("[Diag] O-2p  weight(active):", [float(w_o2p[i])  for i in active_idx])

if (mol.spin == 0) and (nelecas % 2 == 1):
    print("! Warning: singlet but nelecas is odd. Consider tweaking K_window / max_norb or set nelecas explicitly.")

# ---------- 5) CASCI (active-space FCI) ----------
MC = mcscf.CASCI if mol.spin == 0 else mcscf.UCASCI
mycas = MC(mf, ncas=ncas, nelecas=nelecas)

# Tighten FCI solver
mycas.fcisolver = (fci.direct_spin1.FCI(mol) if mol.spin == 0 else fci.direct_uhf.FCI(mol))
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200

# Move chosen MOs to the CAS block front and run
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)

print(f"[CASCI] E(CASCI) = {mycas.e_tot:.12f} Eh   (variationally ≤ E(HF) expected)")
mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp41r48lrq
max_memory 20000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19504610e+00, -5.09687059e-01,  1.36593946e-01, ...,
          1.83871051e-03,  3.39442011e-05,  4.06537561e-15],
        [-5.09687059e-01,  1.33672474e+00, -4.28692661e-01, ...,
         -4.90801101e-03, -1.14615596e-03, -1.44875138e-14],
        [ 1.36593946e-01, -4.28692661e-01,  1.16490653e+00, ...,
          9.25346531e-03,  2.46346323e-03,  1.81430512e-14],
        ...,
        [ 1.83871051e-03, -4.90801101e-03,  9.25346531e-03, ...,
          2.22917795e-02, -1.34539075e-02, -2.54190211e-13],
        [ 3.39442011e-05, -1.14615596e-03,  2.46346323e-03, ...,
         -1.34539075e-02,  8.53245457e-02,  2.11431480e-13],
        [ 4.06537561e-15, -1.44875138e-14,  1.81430512e-14, ...,
         -2.54190211e-13,  2.11431480e-13,  3.35052044e-02]]),
 array([[ 1.19504610e+00, -5.09687059e-01,  1.36593946e-01, ...,
          1.83871051e-03,  3.39442011e-05,  4.06537561e-15],
        [-5.09687059e-01,  1.33672474e+00, -4.28692661e-01, ...,
         -4.90801101e-03, -1.14615596e

In [1]:
# ===== LiCoO2 GAS-PHASE CASCI ladder (pure CI-based, no orbital opt) =====
# - Basis: STO-3G (fast scan)
# - Active-space build: HOMO±K window + Co-3d / O-2p AO→MO weights
# - Auto shrink if FCI dimension too large
# - Try several max_norb values and report the best CASCI energy

import numpy as np
from math import comb
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry: gas-phase cluster from given params ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule (gas phase), basis, spin ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"     # gas-phase model, minimal basis for CAS scan speed
mol.charge = 0
mol.spin   = 0           # singlet (low-spin Co3+ 가정). 필요시 바꾸면 UCASCI 사용됨
mol.max_memory = 20000
mol.build()

# ---------- 2) SCF (tight) ----------
mf = scf.RHF(mol) if mol.spin == 0 else scf.UHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) AO→MO weights for Co-3d / O-2p ----------
def ao_ids_by_atom_L(mol, atom_sym: str, L_letter: str):
    ids = []
    for i, lab in enumerate(mol.ao_labels()):  # e.g. "Co 1 3dxy", "O 2 2px"
        parts = lab.split()
        atom_ok = parts[0].lower().startswith(atom_sym.lower())
        L_ok = (' '+L_letter) in (' '+lab) or parts[-1].lower().startswith(L_letter)
        if atom_ok and L_ok:
            ids.append(i)
    return ids

co_d_ao = ao_ids_by_atom_L(mol, "Co", "d")
o_p_ao  = ao_ids_by_atom_L(mol, "O",  "p")

# RHF: single mo_coeff. UHF면 alpha 계수로 가중치 산출
mo = mf.mo_coeff if mol.spin == 0 else mf.mo_coeff[0]
nmo = mo.shape[1]

def proj_weight(ao_list):
    if not ao_list:
        return np.zeros(nmo)
    sub = mo[ao_list, :]
    return np.sum(sub**2, axis=0)

w_co3d = proj_weight(co_d_ao)
w_o2p  = proj_weight(o_p_ao)

# ---------- 4) Active-space builder (HOMO±K + scoring + auto-shrink) ----------
occ = mf.mo_occ if np.ndim(mf.mo_occ)==1 else 0.5*(mf.mo_occ[0]+mf.mo_occ[1])
occ = np.asarray(occ)
homo = np.where(occ>1.0)[0].max() if np.any(occ>1.0) else (occ.size//2)

K_window   = 12          # candidate window width around HOMO (±K)
low_occ, high_occ = 0.05, 1.95  # light occupancy gate
max_detdim = 5_000_000   # rough cap on determinant count (adjust to your RAM)

proximity = np.array([-abs(i - homo) for i in range(nmo)], dtype=float)
score = w_co3d + 0.5*w_o2p + 0.01*proximity  # simple linear scorer

cand = list(range(max(0, homo-K_window), min(nmo, homo+K_window+1)))
cand_sorted = sorted(cand, key=lambda i: score[i], reverse=True)

def est_dim(norb, na, nb):
    try:
        return comb(norb, na) * comb(norb, nb)
    except ValueError:
        return float('inf')

def build_active(max_norb, occ, score, cand_sorted, singlet=True):
    # 1) 상위 score에서 max_norb만큼 추출
    active = sorted(cand_sorted[:max_norb])
    # 2) 완충/완빈은 가급적 제외
    filtered = [i for i in active if (occ[i] > low_occ and occ[i] < high_occ)]
    if filtered:
        active = sorted(filtered)
    # 3) 전자수 산정
    nele = int(round(float(np.sum(occ[active]))))
    # 4) singlet이면 짝수로 유도(가장 경계 MO 제거)
    if singlet and (nele % 2 == 1) and len(active) > 2:
        worst = min(active, key=lambda i: abs(occ[i]-1.0))
        active.remove(worst)
        nele = int(round(float(np.sum(occ[active]))))
    return active, nele

# ladder: 여러 크기에서 CASCI 시도
ladder_max_norb = [10, 12, 14]  # 필요시 16 추가 (RAM 여유 시)
results = []

for max_norb in ladder_max_norb:
    act, nele = build_active(max_norb, occ, score, cand_sorted, singlet=(mol.spin==0))
    ncas = len(act)
    nalpha = nele // 2
    nbeta  = nele - nalpha
    # FCI dimension 확인 & 과하면 score 낮은 것부터 제거
    while est_dim(ncas, nalpha, nbeta) > max_detdim and ncas > 2:
        worst = min(act, key=lambda i: score[i])
        act.remove(worst)
        ncas = len(act)
        nele = int(round(float(np.sum(occ[act]))))
        nalpha = nele // 2
        nbeta  = nele - nalpha

    print(f"\n[Pick] max_norb={max_norb} → ncas={ncas}, nele={nele}, est dim≈{est_dim(ncas,nalpha,nbeta):,}")
    print("[idx]", act)

    # ---------- 5) CASCI (pure CI in active space) ----------
    MC = mcscf.CASCI if mol.spin == 0 else mcscf.UCASCI
    mycas = MC(mf, ncas=ncas, nelecas=nele)

    # FCI solver tighten
    mycas.fcisolver = fci.direct_spin1.FCI(mol) if mol.spin==0 else fci.direct_uhf.FCI(mol)
    mycas.fcisolver.conv_tol  = 1e-10
    mycas.fcisolver.max_cycle = 200

    # 정렬 후 실행 (언패킹 X)
    mo_sorted = mycas.sort_mo(act, mf.mo_coeff)
    mycas.kernel(mo_coeff=mo_sorted)

    print(f"[CASCI] E = {mycas.e_tot:.12f} Eh   (expect ≤ E(HF))")
    results.append((max_norb, ncas, nele, mycas.e_tot))

# ---------- 6) Summary ----------
best = sorted(results, key=lambda x: x[3])[0]
print("\n===== CASCI ladder summary (gas phase, STO-3G) =====")
for r in results:
    print(f"max_norb={r[0]:>2} | ncas={r[1]:>2} | nele={r[2]:>2} | E={r[3]: .12f} Eh")
print(f"\n>> Best (lowest E): max_norb={best[0]}, ncas={best[1]}, nele={best[2]}, E={best[3]:.12f} Eh")



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp8dezwq0b
max_memory 20000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

In [2]:
# ===== LiCoO2 CASCI with 16 spin orbitals (ncas=8), gas phase, STO-3G =====
# - Active space: exactly 8 spatial MOs around HOMO (contiguous pick)
# - CI method: CASCI (no orbital optimization)
# - Robust to PySCF versions: don't unpack kernel(); read mycas.e_tot

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry from parameters (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule (gas phase), basis, spin ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"     # requested: minimal basis
mol.charge = 0
mol.spin   = 0           # singlet (low-spin Co3+ 가정). 개방각이면 적절히 바꾸세요.
mol.max_memory = 20000
mol.build()

# ---------- 2) SCF (tight) ----------
mf = scf.RHF(mol) if mol.spin == 0 else scf.UHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) Build active space: exactly 8 spatial MOs around HOMO ----------
# Occupations (RHF or average(UHF))
occ = mf.mo_occ if np.ndim(mf.mo_occ)==1 else 0.5*(mf.mo_occ[0] + mf.mo_occ[1])
occ = np.asarray(occ)
nmo  = occ.size
# HOMO index (safe fallback)
homo = np.where(occ > 1.0)[0].max() if np.any(occ>1.0) else nmo // 2

# Pick contiguous 8 MOs around HOMO: [HOMO-3, HOMO-2, HOMO-1, HOMO, LUMO, LUMO+1, LUMO+2, LUMO+3]
below, above = 4, 4   # total = 8
cand = list(range(max(0, homo-below+1), homo+1)) + list(range(homo+1, min(nmo, homo+1+above)))
# Guard for edges and duplicates
active_idx = sorted(set(cand))[:8]
# If at boundaries and got <8, pad from nearest side
i_left, i_right = homo-4, homo+4
while len(active_idx) < 8 and i_left >= 0:
    if i_left not in active_idx: active_idx = sorted(active_idx + [i_left])
    i_left -= 1
while len(active_idx) < 8 and i_right < nmo:
    if i_right not in active_idx: active_idx = sorted(active_idx + [i_right])
    i_right += 1
active_idx = active_idx[:8]
ncas = len(active_idx)

# Active-space electron count = rounded sum of HF occupations in selected MOs
nelecas = int(round(float(np.sum(occ[active_idx]))))
# For singlet, prefer even electron count; if odd, drop the MO closest to fully 0/2 to fix parity
if mol.spin == 0 and nelecas % 2 == 1 and ncas > 2:
    worst = min(active_idx, key=lambda i: abs(occ[i]-1.0))
    active_idx.remove(worst)
    ncas = len(active_idx)
    nelecas = int(round(float(np.sum(occ[active_idx]))))

print(f"[Active space] ncas={ncas} (8 target), nelecas={nelecas}, idx={active_idx}")
print("[occ in active]:", [float(occ[i]) for i in active_idx])

# ---------- 4) CASCI (= FCI in the active space) ----------
MC = mcscf.CASCI if mol.spin == 0 else mcscf.UCASCI
mycas = MC(mf, ncas=ncas, nelecas=nelecas)
# Use strict FCI solver settings
mycas.fcisolver = fci.direct_spin1.FCI(mol) if mol.spin==0 else fci.direct_uhf.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200

# Reorder so selected MOs form the CAS block, then run
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)   # don't unpack; use attributes

print(f"[CASCI (8 orb = 16 spin)] E = {mycas.e_tot:.12f} Eh")
mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpufw92o5d
max_memory 20000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19495260e+00, -5.09419992e-01,  1.35711717e-01, ...,
         -7.82814819e-03, -2.23311552e-03, -1.26382991e-13],
        [-5.09419992e-01,  1.33597272e+00, -4.26096651e-01, ...,
          2.71179548e-02,  6.22632089e-03,  3.30197047e-13],
        [ 1.35711717e-01, -4.26096651e-01,  1.15711524e+00, ...,
         -5.10399515e-02, -1.26614011e-02, -1.39322531e-12],
        ...,
        [-7.82814819e-03,  2.71179548e-02, -5.10399515e-02, ...,
          8.29821420e-01,  1.56359450e-01, -1.21197462e-12],
        [-2.23311552e-03,  6.22632089e-03, -1.26614011e-02, ...,
          1.56359450e-01,  1.54083518e-01,  1.11122030e-11],
        [-1.26382991e-13,  3.30197047e-13, -1.39322531e-12, ...,
         -1.21197462e-12,  1.11122030e-11,  7.39048710e-04]]),
 array([[ 1.19495261e+00, -5.09420022e-01,  1.35711794e-01, ...,
         -7.82801618e-03, -2.23309382e-03,  9.97693803e-14],
        [-5.09420022e-01,  1.33597281e+00, -4.26096888e-01, ...,
          2.71175426e-02,  6.22625208e

In [3]:
# ===== LiCoO2 CASCI with HOMO(8) + LUMO(4) => ncas=12, gas phase, STO-3G =====
# - Active space: HOMO-7..HOMO (8개) + LUMO..LUMO+3 (4개) = 12 spatial MOs
# - CI method: CASCI (오비탈 고정)
# - PySCF 버전 안전: kernel 반환값 언패킹하지 않음

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry from parameters (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule (gas phase), basis, spin ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"     # 요청: STO-3G
mol.charge = 0
mol.spin   = 0           # singlet 가정 (필요시 변경)
mol.max_memory = 20000
mol.build()

# ---------- 2) SCF (tight) ----------
mf = scf.RHF(mol) if mol.spin == 0 else scf.UHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) Build active space: HOMO 8 + LUMO 4 ----------
# 점유수(알파/베타 평균)로 HOMO/LUMO 식별
occ = mf.mo_occ if np.ndim(mf.mo_occ)==1 else 0.5*(mf.mo_occ[0] + mf.mo_occ[1])
occ = np.asarray(occ)
nmo  = occ.size

# HOMO index (occ > 1.0 중 가장 큰 인덱스; 안전망 포함)
if np.any(occ > 1.0):
    homo = np.where(occ > 1.0)[0].max()
else:
    homo = nmo // 2  # fallback

# 원하는 선택: HOMO-7..HOMO (8개) + LUMO..LUMO+3 (4개)
homo_block = list(range(max(0, homo-7), homo+1))  # 포함: homo-7 .. homo
lumo_start = min(homo+1, nmo-1)
lumo_end   = min(homo+4, nmo-1)                   # 포함: lumo_start .. lumo_start+3
lumo_block = list(range(lumo_start, lumo_end+1))

active_idx = sorted(set(homo_block + lumo_block))
# 경계에서 12개가 안 되면 근처에서 보충 (희귀 케이스)
while len(active_idx) < 12:
    # 왼쪽/오른쪽에서 하나씩 보충
    left  = max(0, min(active_idx) - 1)
    right = min(nmo-1, max(active_idx) + 1)
    if left not in active_idx:  active_idx.append(left)
    if len(active_idx) < 12 and right not in active_idx: active_idx.append(right)
    active_idx = sorted(set(active_idx))
# 혹시 12개 초과되면 앞에서 12개만
active_idx = active_idx[:12]
ncas = len(active_idx)

print(f"[Active idx] {active_idx}  (ncas={ncas})")
print("[occ in active]:", [float(occ[i]) for i in active_idx])

# 활성공간 전자수: 선택 MO들의 평균 점유수 합을 반올림
nelecas = int(round(float(np.sum(occ[active_idx]))))
# singlet이면 짝수 권장 → 홀수면 경계 MO 하나 제거해서 짝수 맞춤
if mol.spin == 0 and nelecas % 2 == 1 and ncas > 2:
    # 0 또는 2에 가장 가까운(= 덜 중요한) MO를 하나 제거
    worst = min(active_idx, key=lambda i: abs(occ[i]-1.0))
    active_idx.remove(worst)
    ncas = len(active_idx)
    nelecas = int(round(float(np.sum(occ[active_idx]))))

print(f"[Active space] ncas={ncas} (target 12), nelecas={nelecas}")

# ---------- 4) CASCI (= FCI in the chosen 12 orbitals) ----------
MC = mcscf.CASCI if mol.spin == 0 else mcscf.UCASCI
mycas = MC(mf, ncas=ncas, nelecas=nelecas)

# FCI solver 설정(수렴 강화)
mycas.fcisolver = fci.direct_spin1.FCI(mol) if mol.spin==0 else fci.direct_uhf.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200

# 활성 MO를 앞(CAS 블록)으로 정렬 후 실행
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)   # 언패킹 금지

print(f"[CASCI] E(CASCI, 12orb=24spin) = {mycas.e_tot:.12f} Eh")
mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpew12fexq
max_memory 20000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19494413e+00, -5.09393976e-01,  1.35644868e-01, ...,
         -7.91667193e-03, -2.13377106e-03, -3.47002713e-12],
        [-5.09393976e-01,  1.33589125e+00, -4.25901994e-01, ...,
          2.73374642e-02,  6.18666613e-03,  9.73840009e-12],
        [ 1.35644868e-01, -4.25901994e-01,  1.15651397e+00, ...,
         -5.20752001e-02, -1.00277208e-02, -3.21359744e-11],
        ...,
        [-7.91667193e-03,  2.73374642e-02, -5.20752001e-02, ...,
          8.27186147e-01,  1.61623083e-01, -1.63327725e-10],
        [-2.13377106e-03,  6.18666613e-03, -1.00277208e-02, ...,
          1.61623083e-01,  1.02961417e-01, -1.81197427e-11],
        [-3.47002713e-12,  9.73840009e-12, -3.21359744e-11, ...,
         -1.63327725e-10, -1.81197427e-11,  5.01828884e-03]]),
 array([[ 1.19494415e+00, -5.09394006e-01,  1.35645012e-01, ...,
         -7.91624642e-03, -2.13485498e-03,  3.49384489e-12],
        [-5.09394006e-01,  1.33589132e+00, -4.25902346e-01, ...,
          2.73361394e-02,  6.18884500e

In [4]:
# LiCoO2 ground-state energy via CCSD(T)
# - Gas phase
# - Basis: STO-3G (변경하려면 BASIS 변수만 바꾸세요)
# - 자동으로 RHF→CCSD(T) (spin=0) / UHF→UCCSD(T) (open-shell) 선택

from pyscf import gto, scf, cc
import numpy as np

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""
print("[geom]\n", geom)

# ---------- 1) Molecule / basis / spin ----------
BASIS = "sto-3g"     # 필요시 'def2-SVP' 등으로 변경
CHARGE = 0
SPIN2S  = 0          # singlet 가정; 개방각이면 1,2..로 조정

mol = gto.Mole()
mol.atom  = geom
mol.basis = BASIS
mol.charge = CHARGE
mol.spin   = SPIN2S
mol.max_memory = 24000  # MB (환경에 맞춰 조정)
mol.build()

# ---------- 2) SCF ----------
if mol.spin == 0:
    mf = scf.RHF(mol)
else:
    mf = scf.UHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) CCSD(T) ----------
if mol.spin == 0:
    mycc = cc.CCSD(mf)      # RHF 참조
else:
    mycc = cc.UCCSD(mf)     # UHF 참조 (open-shell)

mycc.conv_tol  = 1e-8
mycc.max_cycle = 200
mycc = mycc.run()           # CCSD 수렴
E_CCSD = mycc.e_tot
print(f"[CCSD] E = {E_CCSD:.12f} Eh")

# perturbative (T) triples correction
# PySCF: ccsd_t()는 (T) 보정 에너지 ΔE_(T)만 반환
dE_T = mycc.ccsd_t()
E_CCSD_T = E_CCSD + dE_T
print(f"[(T)]  ΔE = {dE_T:.12f} Eh")
print(f"[CCSD(T)] E = {E_CCSD_T:.12f} Eh")

# ---------- 4) Sanity checks ----------
# 일반적으로 E(CCSD) ≤ E(HF), E[CCSD(T)] ≤ E[CCSD] 가 기대됩니다.

[geom]
 
Co   0.00000000   0.00000000   0.00000000
O    1.92200000   0.00000000   0.00000000
O    -0.14210199   1.91673969   0.00000000
Li   2.07686307   -2.08886730   0.00000000



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp6fd9v_b_
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO 

In [6]:
# ===== LiCoO2 CASCI (HOMO 8 + LUMO 4) with HF-reference-safe nelecas =====
# - Gas phase, STO-3G
# - Active space: HOMO-7..HOMO (8개) + LUMO..LUMO+3 (4개) = ncas=12
# - Key fix: nelecas = 2 * (# of HF-doubly-occupied orbitals inside active space)
# - Expectation: E(CASCI) <= E(HF)

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule (gas), basis, spin ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"   # 요청: STO-3G
mol.charge = 0
mol.spin   = 0         # singlet 가정 (개방각이면 UHF+UCASCI로 변경 필요)
mol.max_memory = 24000
mol.build()

# ---------- 2) SCF (tight) ----------
mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 3) Active space: HOMO 8 + LUMO 4 (총 12개) ----------
occ = np.asarray(mf.mo_occ)  # RHF 점유수 (각 MO당 0/2)
nmo = occ.size
if np.any(occ > 1.0):
    homo = np.where(occ > 1.0)[0].max()
else:
    homo = nmo // 2  # 안전망

# HOMO-7..HOMO (8개)
homo_block = list(range(max(0, homo-7), homo+1))
# LUMO..LUMO+3 (4개)
lumo_start = min(homo+1, nmo-1)
lumo_end   = min(homo+4, nmo-1)
lumo_block = list(range(lumo_start, lumo_end+1))

active_idx = sorted(set(homo_block + lumo_block))
# 경계에서 부족하면 양옆으로 보충
while len(active_idx) < 12 and (min(active_idx) > 0 or max(active_idx) < nmo-1):
    left  = max(0, min(active_idx)-1)
    right = min(nmo-1, max(active_idx)+1)
    if left not in active_idx:  active_idx.append(left)
    if len(active_idx) < 12 and right not in active_idx: active_idx.append(right)
    active_idx = sorted(set(active_idx))
# 초과되면 앞에서 12개만
active_idx = active_idx[:12]
ncas = len(active_idx)

# ---------- 4) 핵심 FIX: nelecas를 HF 기준으로 '명시' ----------
# singlet(RHF)에서 active에 들어온 'HF에서 점유수=2'인 오비탈 개수 × 2
n_doubly_in_active = sum(occ[i] > 1.0 for i in active_idx)
nelecas = 2 * n_doubly_in_active
assert nelecas % 2 == 0, "Singlet에서는 nelecas가 짝수여야 합니다."

print(f"[Active idx] {active_idx}  (ncas={ncas})")
print("[occ(active)]", [float(occ[i]) for i in active_idx])
print(f"[Fix] nelecas = 2 * {n_doubly_in_active} = {nelecas}")

# ---------- 5) CASCI (= FCI in active space) ----------
MC = mcscf.CASCI
mycas = MC(mf, ncas=ncas, nelecas=nelecas)

# FCI solver 설정(엄격 수렴)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200

# 선택한 MO를 CAS 블록 앞으로 정렬 후 실행
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)   # 반환값 언패킹 금지 (버전 호환)

E_cas = mycas.e_tot
print(f"[CASCI] E = {E_cas:.12f} Eh  (expected ≤ E(HF))")

# 간단 sanity
if E_cas > mf.e_tot + 1e-8:
    print("! Warning: E(CASCI) > E(HF). active_idx/nelecas 또는 참조파를 다시 점검하세요.")

mycas.analyze()




******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp21y1k40_
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19494413e+00, -5.09393976e-01,  1.35644868e-01, ...,
         -7.91667193e-03, -2.13377106e-03, -3.47002713e-12],
        [-5.09393976e-01,  1.33589125e+00, -4.25901994e-01, ...,
          2.73374642e-02,  6.18666613e-03,  9.73840009e-12],
        [ 1.35644868e-01, -4.25901994e-01,  1.15651397e+00, ...,
         -5.20752001e-02, -1.00277208e-02, -3.21359744e-11],
        ...,
        [-7.91667193e-03,  2.73374642e-02, -5.20752001e-02, ...,
          8.27186147e-01,  1.61623083e-01, -1.63327725e-10],
        [-2.13377106e-03,  6.18666613e-03, -1.00277208e-02, ...,
          1.61623083e-01,  1.02961417e-01, -1.81197427e-11],
        [-3.47002713e-12,  9.73840009e-12, -3.21359744e-11, ...,
         -1.63327725e-10, -1.81197427e-11,  5.01828884e-03]]),
 array([[ 1.19494415e+00, -5.09394006e-01,  1.35645012e-01, ...,
         -7.91624642e-03, -2.13485498e-03,  3.49384489e-12],
        [-5.09394006e-01,  1.33589132e+00, -4.25902346e-01, ...,
          2.73361394e-02,  6.18884500e

In [7]:
# ===== LiCoO2 CASCI (STO-3G, gas) — HF 참조 포함 보증판 =====
# - Active space: "마지막 8개 점유(HF) + 처음 4개 비점유(HF)" = 총 12 공간오비탈
# - nelecas = 2 * (#active에서 점유된 오비탈 개수)  ← HF determinant 포함 보장
# - PySCF 버전 안전: kernel() 반환값 언패킹하지 않음

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)
Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule/SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0           # RHF/singlet 가정 (개방각이면 UHF+UCASCI로 변경)
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) 점유수 '기반' 활성공간 선택 (오해 소지 제거) ----------
occ = np.asarray(mf.mo_occ)           # RHF에서는 0 또는 2
nmo = occ.size

occ_idx = [i for i in range(nmo) if occ[i] > 1.0]  # 점유(=2)
vir_idx = [i for i in range(nmo) if occ[i] < 1.0]  # 비점유(=0)

# 안전장치: 최소 개수 확인
assert len(occ_idx) >= 8,  "점유 오비탈이 8개 미만이라면 시스템/기저를 확인하세요."
assert len(vir_idx) >= 4,  "비점유 오비탈이 4개 미만이라면 시스템/기저를 확인하세요."

# 마지막 8개 점유 + 처음 4개 비점유
occ_sel  = occ_idx[-8:]
vir_sel  = vir_idx[:4]
active_idx = sorted(occ_sel + vir_sel)
ncas = len(active_idx)

# 핵심: nelecas를 '명시' — active 내 점유 오비탈 개수 × 2
n_doubly_in_active = len(occ_sel)     # = 8
nelecas = 2 * n_doubly_in_active      # = 16
print(f"[Active] ncas={ncas}, nelecas={nelecas}")
print(f"         occ_sel={occ_sel}")
print(f"         vir_sel={vir_sel}")

# 진단 출력
sum_occ_active = float(np.sum(occ[active_idx]))
print(f"[Diag] sum(HF occ in active) = {sum_occ_active:.1f} (참고값)")
ncore = sum(occ[i] > 1.0 and i not in active_idx for i in range(nmo))
print(f"[Diag] n_core(doubly outside active) = {ncore}")

# ---------- 3) CASCI (FCI in active space) ----------
MC = mcscf.CASCI
mycas = MC(mf, ncas=ncas, nelecas=nelecas)

# FCI solver (엄격)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200
mycas.natorb = False              # (기본값) 오비탈 최적화 없음

# 활성 MO를 앞쪽(CAS 블록)으로 재정렬 후 실행
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)   # 언패킹 금지

E_cas = mycas.e_tot
print(f"[CASCI] E = {E_cas:.12f} Eh  (should be ≤ E(HF) within numerical noise)")

# 안전 체크
if E_cas > mf.e_tot + 1e-8:
    print("! Warning: E(CASCI) > E(HF). 비정상입니다.")
    print("  - active_idx가 'HF에서 점유 8 + 비점유 4'인지 재확인")
    print("  - RHF가 맞는지(UHF 아님), spin=0이 맞는지 체크")
    print("  - 혹시라도 sort_mo 인덱스가 MO 순서와 엇갈리지 않았는지 확인")

mycas.analyze()




******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpvzz4x6nx
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19494413e+00, -5.09393976e-01,  1.35644868e-01, ...,
         -7.91667193e-03, -2.13377106e-03, -3.47002713e-12],
        [-5.09393976e-01,  1.33589125e+00, -4.25901994e-01, ...,
          2.73374642e-02,  6.18666613e-03,  9.73840009e-12],
        [ 1.35644868e-01, -4.25901994e-01,  1.15651397e+00, ...,
         -5.20752001e-02, -1.00277208e-02, -3.21359744e-11],
        ...,
        [-7.91667193e-03,  2.73374642e-02, -5.20752001e-02, ...,
          8.27186147e-01,  1.61623083e-01, -1.63327725e-10],
        [-2.13377106e-03,  6.18666613e-03, -1.00277208e-02, ...,
          1.61623083e-01,  1.02961417e-01, -1.81197427e-11],
        [-3.47002713e-12,  9.73840009e-12, -3.21359744e-11, ...,
         -1.63327725e-10, -1.81197427e-11,  5.01828884e-03]]),
 array([[ 1.19494415e+00, -5.09394006e-01,  1.35645012e-01, ...,
         -7.91624642e-03, -2.13485498e-03,  3.49384489e-12],
        [-5.09394006e-01,  1.33589132e+00, -4.25902346e-01, ...,
          2.73361394e-02,  6.18884500e

In [8]:
# ===== LiCoO2 CASCI (active: 16 spin orbitals = 8 spatial) =====
# - Gas phase, STO-3G
# - Active MOs: last 4 occupied (HF) + first 4 virtual (HF) = 8 spatial
# - Ensures HF reference is inside the CAS (E_CASCI <= E_HF expected)
# - Minimal & robust: no unpack of kernel() return

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
import numpy as np
theta = np.deg2rad(94.24)

Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0           # singlet (RHF) 가정
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) Build 8-orbital active space: occ(4) + virt(4) ----------
occ = np.asarray(mf.mo_occ)  # RHF: 0 or 2 per MO
nmo = occ.size
# indices of occupied / virtual MOs by HF
occ_idx = [i for i in range(nmo) if occ[i] > 1.0]   # doubly-occupied
vir_idx = [i for i in range(nmo) if occ[i] < 1.0]   # unoccupied
assert len(occ_idx) >= 4 and len(vir_idx) >= 4, "STO-3G에서 최소한 4/4가 나와야 합니다."

occ_sel = occ_idx[-4:]       # last 4 occupied
vir_sel = vir_idx[:4]        # first 4 virtual
active_idx = sorted(occ_sel + vir_sel)
ncas = len(active_idx)       # = 8

# electrons in CAS: 2 * (# occupied inside active) → includes HF reference
nelecas = 2 * len(occ_sel)   # = 8 electrons
print(f"[Active] ncas={ncas} (target 8), nelecas={nelecas}")
print(f"         occ_sel={occ_sel}")
print(f"         vir_sel={vir_sel}")

# ---------- 3) CASCI (= FCI in the 8 orbitals) ----------
mycas = mcscf.CASCI(mf, ncas=ncas, nelecas=nelecas)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200

# reorder so the selected MOs form the CAS block; then run
mo_sorted = mycas.sort_mo(active_idx, mf.mo_coeff)
mycas.kernel(mo_coeff=mo_sorted)   # do not unpack

E_cas = mycas.e_tot
print(f"[CASCI] E(8 spatial = 16 spin) = {E_cas:.12f} Eh  (should be ≤ E(HF))")

# quick sanity
if E_cas > mf.e_tot + 1e-8:
    print("! Warning: E(CASCI) > E(HF). 비정상입니다. RHF 여부, active_idx, nelecas를 다시 확인하세요.")

mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp42q1cfet
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19495260e+00, -5.09419992e-01,  1.35711717e-01, ...,
         -7.82814819e-03, -2.23311552e-03, -1.26382991e-13],
        [-5.09419992e-01,  1.33597272e+00, -4.26096651e-01, ...,
          2.71179548e-02,  6.22632089e-03,  3.30197047e-13],
        [ 1.35711717e-01, -4.26096651e-01,  1.15711524e+00, ...,
         -5.10399515e-02, -1.26614011e-02, -1.39322531e-12],
        ...,
        [-7.82814819e-03,  2.71179548e-02, -5.10399515e-02, ...,
          8.29821420e-01,  1.56359450e-01, -1.21197462e-12],
        [-2.23311552e-03,  6.22632089e-03, -1.26614011e-02, ...,
          1.56359450e-01,  1.54083518e-01,  1.11122030e-11],
        [-1.26382991e-13,  3.30197047e-13, -1.39322531e-12, ...,
         -1.21197462e-12,  1.11122030e-11,  7.39048710e-04]]),
 array([[ 1.19495261e+00, -5.09420022e-01,  1.35711794e-01, ...,
         -7.82801618e-03, -2.23309382e-03,  9.97693803e-14],
        [-5.09420022e-01,  1.33597281e+00, -4.26096888e-01, ...,
          2.71175426e-02,  6.22625208e

In [9]:
# ===== LiCoO2 CASCI (8 spatial / 16 spin) — robust, HF-ref guaranteed =====
# - Gas phase, STO-3G
# - Active MOs: last 4 HF-occupied + first 4 HF-virtual  (= 8 spatial)
# - Key: explicit (nα, nβ) = (4, 4)  AND manual MO reordering -> [Core | Active | Virtual]
# - Expectation: E(CASCI) <= E(HF) (up to ~1e-7 Eh numerical noise)

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)
Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0        # singlet (RHF)
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) Pick Active: last 4 occupied + first 4 virtual (by HF occupations) ----------
occ = np.asarray(mf.mo_occ)      # RHF: exactly 0 or 2
nmo = occ.size

occ_idx = [i for i in range(nmo) if occ[i] > 1.5]  # robust thresholding
vir_idx = [i for i in range(nmo) if occ[i] < 0.5]

assert len(occ_idx) >= 4 and len(vir_idx) >= 4, "Active 만들 점유/비점유 오비탈 개수가 부족합니다."

occ_sel  = occ_idx[-4:]          # last 4 occupied
vir_sel  = vir_idx[:4]           # first 4 virtual
active   = sorted(occ_sel + vir_sel)
ncas     = len(active)           # 8

# Core/Virtual (outside active) 분리
core_idx = [i for i in occ_idx if i not in occ_sel]
virt_rest_idx = [i for i in vir_idx if i not in vir_sel]

# 최종 재배열 순서: [Core | Active | VirtualRest]
new_order = core_idx + active + virt_rest_idx
assert len(new_order) == nmo and len(set(new_order)) == nmo

# 수동 재배열된 MO 계수
C = mf.mo_coeff[:, new_order]

# 활성 블록의 시작/끝 인덱스 (재배열된 기준)
ncore = len(core_idx)
act_slice = slice(ncore, ncore + ncas)

print(f"[Blocks] ncore={ncore}, ncas={ncas}, nvirt_rest={len(virt_rest_idx)}")
print(f"[Active idx (orig)] {active}")
print(f"[Active idx (reordered span)] {act_slice.start}..{act_slice.stop-1}")

# ---------- 3) Active electrons: explicit (nα, nβ) ----------
# last-4 occupied → α=4, β=4  (총 8e in active)
nelecas_tuple = (4, 4)  # using tuple prevents any ambiguity
print(f"[nelecas] nα={nelecas_tuple[0]}, nβ={nelecas_tuple[1]}  (total {sum(nelecas_tuple)})")

# ---------- 4) CASCI (FCI in active) ----------
mycas = mcscf.CASCI(mf, ncas=ncas, nelecas=nelecas_tuple)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200
mycas.natorb = False              # no orbital optimization; pure CASCI
mycas.canonicalization = False    # avoid any re-canonicalization shuffles

# 핵심: 이미 [Core|Active|Virt]로 재배열된 C를 그대로 사용
mycas.kernel(mo_coeff=C)

E_cas = mycas.e_tot
print(f"[CASCI] E(8 spatial / 16 spin) = {E_cas:.12f} Eh")

# ---------- 5) Sanity ----------
tol = 1e-7
if E_cas > mf.e_tot + tol:
    print("! Still E(CASCI) > E(HF). 이 경우는 아주 예외적입니다.")
    print("  - RHF가 실제로 맞는지(개방각 필요? → mol.spin 조정 + UHF+UCASCI)")
    print("  - 재배열된 C에서 Active 블록 위치가 올바른지 (위 인덱스 출력 확인)")
    print("  - PySCF 버전 이슈일 수 있어, 'nelecas=8' (정수) 대신 (4,4) 튜플 사용은 이미 반영함.")
    print("  - 마지막 수단: CASSCF로 0 macro step (오비탈 고정) 또는 1~2 NO 반복으로 테스트")
else:
    print("✓ Variational OK: E(CASCI) ≤ E(HF).")



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpsp60g4pv
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

In [10]:
# ===== LiCoO2 CASCI (HOMO: Co/O 8, LUMO: 4) — gas, STO-3G =====
# - Active: 12 spatial MOs = 8 occupied (Co/O-like) + 4 virtual (prefer Co/O-like)
# - Ensures HF reference inside CAS: nelecas = (8, 8)
# - Manual MO reordering to [Core | Active | VirtualRest]
# - Pure CASCI (no orbital optimization)

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)
Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0               # singlet (RHF)
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) AO→MO weights for Co/O character ----------
def ao_ids_for_atom(mol, atom_sym: str):
    ids = []
    for i, lab in enumerate(mol.ao_labels()):  # ex) "Co 1 3dxy"
        if lab.split()[0].lower().startswith(atom_sym.lower()):
            ids.append(i)
    return ids

co_ao = ao_ids_for_atom(mol, "Co")
o_ao  = ao_ids_for_atom(mol, "O")  # 모든 O 포함 (p 위주 제한 없이 Co/O 전체 성격)

Cmo = mf.mo_coeff                 # RHF MO
def weight_from_ao(ao_idx):
    if not ao_idx: return np.zeros(Cmo.shape[1])
    sub = Cmo[ao_idx, :]          # (n_sel_AO, nmo)
    return np.sum(sub**2, axis=0) # per-MO sum of squares

w_co = weight_from_ao(co_ao)
w_o  = weight_from_ao(o_ao)
w_coo = w_co + w_o                # "Co 또는 O 성격" 가중치

# ---------- 3) Split occupied/virtual and pick HOMO(8) + LUMO(4) with Co/O preference ----------
occ = np.asarray(mf.mo_occ)       # RHF: 0 or 2
nmo = occ.size

occ_idx = [i for i in range(nmo) if occ[i] > 1.5]   # occupied (≈2)
vir_idx = [i for i in range(nmo) if occ[i] < 0.5]   # virtual  (≈0)
assert len(occ_idx) >= 8 and len(vir_idx) >= 4, "HF 점유/비점유 MO 수가 부족합니다."

# HOMO 기준 정렬: 에너지 차순 가정 → 인덱스가 클수록 HOMO/LUMO에 가까움
# 우선순위: (1) Co/O weight 높음, (2) HOMO/LUMO에 가까움
def pick_from(indices, how_many, prefer_weight, reverse_close=True):
    # reverse_close=True: 큰 인덱스 우선(HOMO/LUMO 가까움)
    # 점수 = a*weight + b*(정규화된 인덱스)
    if not indices: return []
    idx_arr = np.array(indices)
    w = prefer_weight[idx_arr]
    # 인덱스 근접도(0~1 스케일)
    rank = (idx_arr - idx_arr.min()) / max(1, (idx_arr.max() - idx_arr.min()))
    score = w + 0.01*(rank if reverse_close else (1-rank))
    order = list(idx_arr[np.argsort(-score)])
    return order[:how_many]

# 8개의 점유 MO (Co/O 성격 우선, HOMO 근접 우선)
occ_sel_coo = pick_from(occ_idx, how_many=8, prefer_weight=w_coo, reverse_close=True)
# 부족하면 순수 HOMO 근접으로 보충
if len(occ_sel_coo) < 8:
    need = 8 - len(occ_sel_coo)
    remain = [i for i in occ_idx if i not in occ_sel_coo]
    occ_sel_coo += remain[-need:]
occ_sel_coo = sorted(occ_sel_coo)

# 4개의 비점유 MO (Co/O 성격 우선, LUMO 근접 우선)
vir_sel_coo = pick_from(vir_idx, how_many=4, prefer_weight=w_coo, reverse_close=False)
# 부족하면 순수 LUMO 근접으로 보충
if len(vir_sel_coo) < 4:
    need = 4 - len(vir_sel_coo)
    remain = [i for i in vir_idx if i not in vir_sel_coo]
    vir_sel_coo += remain[:need]
vir_sel_coo = sorted(vir_sel_coo)

active = sorted(occ_sel_coo + vir_sel_coo)
ncas = len(active)               # = 12
print(f"[Active pick] occ(8)={occ_sel_coo}, vir(4)={vir_sel_coo} -> ncas={ncas}")

# ---------- 4) Manual MO reordering: [Core | Active | VirtualRest] ----------
core_idx = [i for i in occ_idx if i not in occ_sel_coo]
virt_rest_idx = [i for i in vir_idx if i not in vir_sel_coo]
new_order = core_idx + active + virt_rest_idx
assert len(new_order) == nmo and len(set(new_order)) == nmo

C_reordered = Cmo[:, new_order]
ncore = len(core_idx)
act_slice = slice(ncore, ncore + ncas)
print(f"[Blocks] ncore={ncore}, active span={act_slice.start}..{act_slice.stop-1}, nvirt_rest={len(virt_rest_idx)}")

# ---------- 5) Electrons in CAS: explicit (nα, nβ) = (8, 8) ----------
nelecas = (8, 8)   # 8 occupied MOs in active → total 16e → (8,8)
print(f"[nelecas] nα={nelecas[0]}, nβ={nelecas[1]}  (total {sum(nelecas)})")

# ---------- 6) CASCI (FCI in the 12 orbitals) ----------
mycas = mcscf.CASCI(mf, ncas=ncas, nelecas=nelecas)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200
mycas.natorb = False
mycas.canonicalization = False

mycas.kernel(mo_coeff=C_reordered)   # already block-ordered
E_cas = mycas.e_tot
print(f"[CASCI] E(12 orb = 24 spin) = {E_cas:.12f} Eh   (variationally ≤ E(HF) expected)")

# Quick sanity
if E_cas > mf.e_tot + 1e-7:
    print("! Warning: E(CASCI) > E(HF). RHF/스핀, active 구성, 재배열을 다시 확인하세요.")

mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpd2azyqc3
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19494887e+00, -5.09379523e-01,  1.35884295e-01, ...,
          3.42310326e-03,  8.79497198e-04, -3.61302032e-16],
        [-5.09379523e-01,  1.33575243e+00, -4.26445000e-01, ...,
         -9.78687452e-03, -3.69534322e-03,  9.91800439e-16],
        [ 1.35884295e-01, -4.26445000e-01,  1.15974660e+00, ...,
          2.16091901e-02,  9.40335636e-03, -4.04253744e-15],
        ...,
        [ 3.42310326e-03, -9.78687452e-03,  2.16091901e-02, ...,
          3.96039868e-02,  9.04386582e-03, -1.62902204e-13],
        [ 8.79497198e-04, -3.69534322e-03,  9.40335636e-03, ...,
          9.04386582e-03,  9.76504695e-02, -8.29016195e-14],
        [-3.61302032e-16,  9.91800439e-16, -4.04253744e-15, ...,
         -1.62902204e-13, -8.29016195e-14,  6.85685137e-02]]),
 array([[ 1.19494887e+00, -5.09379523e-01,  1.35884295e-01, ...,
          3.42310326e-03,  8.79497198e-04, -3.61302032e-16],
        [-5.09379523e-01,  1.33575243e+00, -4.26445000e-01, ...,
         -9.78687452e-03, -3.69534322e

In [11]:
# ===== LiCoO2 CASCI with custom occupied pick: Co 5, O 3, Li 1 (min), total 12 occ + 4 virt =====
# - Gas phase, STO-3G
# - Occupied(“안쪽”): 최소 보장(Co=5, O=3, Li=1) + 가중치/근접도 채움 → 총 12개
# - Virtual: 4개
# - Active = 16 orbitals → RHF singlet에서 nelecas=(12,12)
# - 수동 MO 재배열로 [Core | Active | VirtualRest] 보장 → HF 참조 포함

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)
Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0               # singlet (RHF)
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) AO→MO weights for Co / O / Li ----------
def ao_ids_for_atom(mol, atom_sym: str):
    ids = []
    for i, lab in enumerate(mol.ao_labels()):  # ex) "Co 1 3dxy"
        if lab.split()[0].lower().startswith(atom_sym.lower()):
            ids.append(i)
    return ids

ao_co = ao_ids_for_atom(mol, "Co")
ao_o  = ao_ids_for_atom(mol, "O")
ao_li = ao_ids_for_atom(mol, "Li")

Cmo = mf.mo_coeff  # RHF MO coeffs

def weight_from_ao(ao_idx):
    if not ao_idx: return np.zeros(Cmo.shape[1])
    sub = Cmo[ao_idx, :]          # (n_sel_AO, nmo)
    return np.sum(sub**2, axis=0) # per-MO sum of squares (non-negative)

w_co  = weight_from_ao(ao_co)
w_o   = weight_from_ao(ao_o)
w_li  = weight_from_ao(ao_li)
w_all = w_co + w_o + w_li

# ---------- 3) Occupied / Virtual partition (RHF) ----------
occ = np.asarray(mf.mo_occ)       # RHF: 0 or 2
nmo = occ.size
occ_idx = [i for i in range(nmo) if occ[i] > 1.5]  # doubly occupied
vir_idx = [i for i in range(nmo) if occ[i] < 0.5]  # unoccupied
assert len(occ_idx) >= 12 and len(vir_idx) >= 4, "Occupied/Virtual MO 수가 부족합니다."

# HOMO/LUMO 근접도: 인덱스가 클수록 HOMO/LUMO 근처 (대략)
homo = np.where(occ > 1.5)[0].max()
rank_occ = (np.array(occ_idx) - occ_idx[0]) / max(1, (occ_idx[-1] - occ_idx[0]))
rank_vir = (vir_idx[-1] - np.array(vir_idx)) / max(1, (vir_idx[-1] - vir_idx[0]))  # LUMO 가까울수록 큼

# ---------- 4) “최소 보장”으로 먼저 뽑기: Co 5, O 3, Li 1 ----------
def top_k_from(indices, weight_vec, k, proximity_vec=None, lam=0.01):
    if k <= 0 or not indices: return []
    idx_arr = np.array(indices)
    w = weight_vec[idx_arr]
    prox = np.zeros_like(w) if proximity_vec is None else proximity_vec
    score = w + lam*prox
    order = list(idx_arr[np.argsort(-score)])
    pick = []
    for i in order:
        if i not in pick:
            pick.append(i)
            if len(pick) == k: break
    return pick

# 각 원자 그룹에서 우선 선택 (H가 아니라 Li로 해석)
occ_co_min = 5
occ_o_min  = 3
occ_li_min = 1
target_occ_total = 12
target_vir_total = 4

# 그룹별 후보 (occupied만)
co_occ_cand = [i for i in occ_idx]  # Co/O/Li 분리는 weight로 반영
o_occ_cand  = [i for i in occ_idx]
li_occ_cand = [i for i in occ_idx]

# 근접도 벡터(occupied): HOMO 근접도 = rank_occ
prox_occ = np.zeros(nmo); prox_occ[occ_idx] = rank_occ

sel_occ = []
sel_occ += top_k_from(co_occ_cand, w_co, k=occ_co_min, proximity_vec=prox_occ[co_occ_cand])
sel_occ += top_k_from(o_occ_cand,  w_o,  k=occ_o_min,  proximity_vec=prox_occ[o_occ_cand])
sel_occ += top_k_from(li_occ_cand, w_li, k=occ_li_min, proximity_vec=prox_occ[li_occ_cand])
sel_occ = sorted(set(sel_occ))

# 부족분은 총 12개가 되도록 "종합 가중치 + 근접도"로 채우기
remain_occ_needed = max(0, target_occ_total - len(sel_occ))
if remain_occ_needed > 0:
    remain_pool = [i for i in occ_idx if i not in sel_occ]
    # 종합 점수: w_all + 0.01*근접
    extra = top_k_from(remain_pool, w_all, k=remain_occ_needed, proximity_vec=prox_occ[remain_pool])
    sel_occ = sorted(set(sel_occ + extra))

# 혹시 초과되면 HOMO 근접/가중치 낮은 것부터 잘라내기
if len(sel_occ) > target_occ_total:
    tmp = sel_occ.copy()
    # 낮은 점수(= w_all + 0.01*prox) 순으로 제거
    def score_occ(i): return w_all[i] + 0.01*prox_occ[i]
    tmp_sorted = sorted(tmp, key=lambda i: score_occ(i))  # 낮은 게 먼저
    cut = len(sel_occ) - target_occ_total
    for i in tmp_sorted[:cut]:
        sel_occ.remove(i)
sel_occ = sorted(sel_occ)
assert len(sel_occ) == target_occ_total, f"occupied 선택 개수 불일치: {len(sel_occ)}"

# ---------- 5) Virtual 4개 선택 (Co/O/Li 성격 + LUMO 근접) ----------
prox_vir = np.zeros(nmo); prox_vir[vir_idx] = rank_vir
sel_vir = top_k_from(vir_idx, w_all, k=target_vir_total, proximity_vec=prox_vir[vir_idx])
sel_vir = sorted(sel_vir)
assert len(sel_vir) == target_vir_total, f"virtual 선택 개수 불일치: {len(sel_vir)}"

active = sorted(sel_occ + sel_vir)
ncas = len(active)  # = 16
print(f"[Active pick] occ(12)={sel_occ}")
print(f"               vir(4) ={sel_vir}")
print(f"               ncas   ={ncas}")

# ---------- 6) Manual MO reordering: [Core | Active | VirtualRest] ----------
core_idx = [i for i in occ_idx if i not in sel_occ]
virt_rest_idx = [i for i in vir_idx if i not in sel_vir]
new_order = core_idx + active + virt_rest_idx
assert len(new_order) == nmo and len(set(new_order)) == nmo

C_reordered = Cmo[:, new_order]
ncore = len(core_idx)
act_slice = slice(ncore, ncore + ncas)
print(f"[Blocks] ncore={ncore}, active span={act_slice.start}..{act_slice.stop-1}, nvirt_rest={len(virt_rest_idx)}")

# ---------- 7) Electrons in CAS (singlet RHF): nelecas = (12, 12) ----------
nelecas = (12, 12)   # occupied 12개 → 총 24e → (12,12)
print(f"[nelecas] nα={nelecas[0]}, nβ={nelecas[1]} (total {sum(nelecas)})")

# ---------- 8) CASCI (FCI in the 16 orbitals) ----------
mycas = mcscf.CASCI(mf, ncas=ncas, nelecas=nelecas)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200
mycas.natorb = False
mycas.canonicalization = False

mycas.kernel(mo_coeff=C_reordered)
E_cas = mycas.e_tot
print(f"[CASCI] E(16 orb = 32 spin) = {E_cas:.12f} Eh   (variationally ≤ E(HF) expected)")

if E_cas > mf.e_tot + 1e-7:
    print("! Warning: E(CASCI) > E(HF). RHF 여부, active 구성, 재배열을 확인하세요.")

mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmp3b_ikxsx
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

(array([[ 1.19494841e+00, -5.09378110e-01,  1.35880566e-01, ...,
          3.45008292e-03,  8.90470955e-04, -2.99295035e-16],
        [-5.09378110e-01,  1.33574815e+00, -4.26433573e-01, ...,
         -9.86933971e-03, -3.73168103e-03,  7.85095498e-16],
        [ 1.35880566e-01, -4.26433573e-01,  1.15971719e+00, ...,
          2.18255065e-02,  9.47460104e-03, -3.67828086e-15],
        ...,
        [ 3.45008292e-03, -9.86933971e-03,  2.18255065e-02, ...,
          4.02752948e-02,  9.34144601e-03, -1.67555188e-13],
        [ 8.90470955e-04, -3.73168103e-03,  9.47460104e-03, ...,
          9.34144601e-03,  9.83504124e-02, -8.39752832e-14],
        [-2.99295035e-16,  7.85095498e-16, -3.67828086e-15, ...,
         -1.67555188e-13, -8.39752832e-14,  6.91167510e-02]]),
 array([[ 1.19494841e+00, -5.09378110e-01,  1.35880566e-01, ...,
          3.45008292e-03,  8.90470955e-04, -2.99295035e-16],
        [-5.09378110e-01,  1.33574815e+00, -4.26433573e-01, ...,
         -9.86933971e-03, -3.73168103e

In [12]:
# ===== LiCoO2 CASCI with custom occupied pick: Co 5, O 3, Li 1 (min),
# ===== total 12 occupied + 8 virtual  => Active = 20 orbitals (40 spin) =====
# Gas phase, STO-3G, RHF/singlet. HF reference is guaranteed inside CAS.

import numpy as np
from pyscf import gto, scf, mcscf, fci

# ---------- 0) Geometry (Angstrom) ----------
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)
Co = (0.0, 0.0, 0.0)
O1 = (C, 0.0, 0.0)
O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

geom = f"""
Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
"""

# ---------- 1) Molecule / SCF ----------
mol = gto.Mole()
mol.atom  = geom
mol.basis = "sto-3g"
mol.charge = 0
mol.spin   = 0               # singlet (RHF)
mol.max_memory = 24000
mol.build()

mf = scf.RHF(mol)
mf.conv_tol  = 1e-10
mf.max_cycle = 200
mf.verbose   = 4
mf = mf.run()
print(f"[HF] E(HF) = {mf.e_tot:.12f} Eh")

# ---------- 2) AO→MO weights (Co / O / Li) ----------
def ao_ids_for_atom(mol, atom_sym: str):
    ids = []
    for i, lab in enumerate(mol.ao_labels()):
        if lab.split()[0].lower().startswith(atom_sym.lower()):
            ids.append(i)
    return ids

ao_co = ao_ids_for_atom(mol, "Co")
ao_o  = ao_ids_for_atom(mol, "O")
ao_li = ao_ids_for_atom(mol, "Li")

Cmo = mf.mo_coeff
def weight_from_ao(ao_idx):
    if not ao_idx: return np.zeros(Cmo.shape[1])
    sub = Cmo[ao_idx, :]
    return np.sum(sub**2, axis=0)

w_co  = weight_from_ao(ao_co)
w_o   = weight_from_ao(ao_o)
w_li  = weight_from_ao(ao_li)
w_all = w_co + w_o + w_li

# ---------- 3) Occupied / Virtual split ----------
occ = np.asarray(mf.mo_occ)   # RHF: 0 or 2
nmo = occ.size
occ_idx = [i for i in range(nmo) if occ[i] > 1.5]
vir_idx = [i for i in range(nmo) if occ[i] < 0.5]
assert len(occ_idx) >= 12 and len(vir_idx) >= 8, "HF 점유/비점유 MO 수가 부족합니다."

# HOMO/LUMO 근접도 보정(작게 가중)
homo = np.where(occ > 1.5)[0].max()
rank_occ = (np.array(occ_idx) - occ_idx[0]) / max(1, (occ_idx[-1]-occ_idx[0]))
rank_vir = (vir_idx[-1] - np.array(vir_idx)) / max(1, (vir_idx[-1]-vir_idx[0]))

def top_k_from(indices, weight_vec, k, proximity_vec=None, lam=0.01):
    if k <= 0 or not indices: return []
    idx_arr = np.array(indices)
    w = weight_vec[idx_arr]
    prox = np.zeros_like(w) if proximity_vec is None else proximity_vec
    score = w + lam*prox
    order = list(idx_arr[np.argsort(-score)])
    pick = []
    for i in order:
        if i not in pick:
            pick.append(i)
            if len(pick) == k: break
    return pick

# ---------- 4) Occupied 최소 보장: Co 5, O 3, Li 1 → 총 12로 보충 ----------
occ_co_min, occ_o_min, occ_li_min = 5, 3, 1
target_occ_total = 12
target_vir_total = 8   # ★ 변경: Virtual 8개

prox_occ = np.zeros(nmo); prox_occ[occ_idx] = rank_occ
sel_occ = []
sel_occ += top_k_from(occ_idx, w_co, k=occ_co_min, proximity_vec=prox_occ[occ_idx])
sel_occ += top_k_from(occ_idx, w_o,  k=occ_o_min,  proximity_vec=prox_occ[occ_idx])
sel_occ += top_k_from(occ_idx, w_li, k=occ_li_min, proximity_vec=prox_occ[occ_idx])
sel_occ = sorted(set(sel_occ))

remain = max(0, target_occ_total - len(sel_occ))
if remain > 0:
    pool = [i for i in occ_idx if i not in sel_occ]
    extra = top_k_from(pool, w_all, k=remain, proximity_vec=prox_occ[pool])
    sel_occ = sorted(set(sel_occ + extra))
if len(sel_occ) > target_occ_total:
    # 점수 낮은 것(가중치+근접)부터 제거
    def score_occ(i): return w_all[i] + 0.01*prox_occ[i]
    drop = len(sel_occ) - target_occ_total
    for i in sorted(sel_occ, key=score_occ)[:drop]:
        sel_occ.remove(i)
sel_occ = sorted(sel_occ)
assert len(sel_occ) == target_occ_total, f"occupied 선택 개수 불일치: {len(sel_occ)}"

# ---------- 5) Virtual 8개 선택 (Co/O/Li 성격 + LUMO 근접) ----------
prox_vir = np.zeros(nmo); prox_vir[vir_idx] = rank_vir
sel_vir = top_k_from(vir_idx, w_all, k=target_vir_total, proximity_vec=prox_vir[vir_idx])
sel_vir = sorted(sel_vir)
assert len(sel_vir) == target_vir_total, f"virtual 선택 개수 불일치: {len(sel_vir)}"

active = sorted(sel_occ + sel_vir)
ncas = len(active)   # = 20
print(f"[Active pick] occ(12)={sel_occ}")
print(f"               vir(8) ={sel_vir}")
print(f"               ncas   ={ncas}")

# ---------- 6) Manual MO reordering: [Core | Active | VirtualRest] ----------
core_idx = [i for i in occ_idx if i not in sel_occ]
virt_rest_idx = [i for i in vir_idx if i not in sel_vir]
new_order = core_idx + active + virt_rest_idx
assert len(new_order) == nmo and len(set(new_order)) == nmo

C_reordered = Cmo[:, new_order]
ncore = len(core_idx)
act_slice = slice(ncore, ncore + ncas)
print(f"[Blocks] ncore={ncore}, active span={act_slice.start}..{act_slice.stop-1}, nvirt_rest={len(virt_rest_idx)}")

# ---------- 7) Electrons in CAS (RHF singlet): nelecas = (12, 12) ----------
nelecas = (12, 12)   # occupied 12개 → 총 24e
print(f"[nelecas] nα={nelecas[0]}, nβ={nelecas[1]} (total {sum(nelecas)})")

# ---------- 8) CASCI (FCI in the 20 orbitals) ----------
mycas = mcscf.CASCI(mf, ncas=ncas, nelecas=nelecas)
mycas.fcisolver = fci.direct_spin1.FCI(mol)
mycas.fcisolver.conv_tol  = 1e-10
mycas.fcisolver.max_cycle = 200
mycas.natorb = False
mycas.canonicalization = False

mycas.kernel(mo_coeff=C_reordered)
E_cas = mycas.e_tot
print(f"[CASCI] E(20 orb = 40 spin) = {E_cas:.12f} Eh   (variationally ≤ E(HF) expected)")

if E_cas > mf.e_tot + 1e-7:
    print("! Warning: E(CASCI) > E(HF). RHF/스핀, active 구성, 재배열을 확인하세요.")

mycas.analyze()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-10
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/tmpswzy1z9w
max_memory 24000 MB (current use 0 MB)
Set gradient conv threshold to 1e-05
Initial guess from minao.
init E= -1516.93985411888
  HOMO = -0.0208293579103368  LUMO = 0.0103026659982795
cycle= 1 E= -1504.38315634111  delta_E= 12.6  |g|= 4.26  |ddm|=  5.1
  HOMO = -0.603649570783031  LUMO = -0.305182396879597
cycle= 2 E= -1503.11101759111  delta_E= 1.27  |g|= 2.82  |ddm|=  9.2
  HOMO = 0.094874307462828  LUMO = 0.170035538712697
cycle= 3 E= -1504.81262825333  delta_E= -1.7  |g|= 3.22  |ddm|= 8.91
  HOMO = -0.102189034559719  LUMO = -0.0947655673197845
cycle= 4 E= -1517.86519258485  delt

: 